In [1]:
import pandas as pd
import math

# Read the Excel file
file_path = 'D:\\Y_output.xlsx'
df = pd.read_excel(file_path)

In [2]:
# Find indices where the pattern "Data Content ::" occurs
pattern_indices = df[df.iloc[:, 0].str.startswith('Data Content ::', na=False)].index.tolist()
pattern_indices.insert(0,0)

[0, 9, 20]

In [3]:
# Extract and process data between occurrences of the pattern
row_dict = dict()
for i in range(len(pattern_indices)):
    if i == 0:
        start_idx = pattern_indices[i]
        end_idx = pattern_indices[i + 1] - 1
    elif i < len(pattern_indices)-1:
        start_idx = pattern_indices[i] + 2  # Considering 2 rows after the pattern for data extraction
        end_idx = pattern_indices[i + 1] - 1    # Considering 2 rows before the next pattern for data extraction
    else:
        start_idx = pattern_indices[i] + 2
        end_idx = len(df)-1

    print (start_idx, end_idx)

    if start_idx < end_idx :
        
        extracted_data = df.iloc[start_idx:end_idx + 1]
    
        # Remove rows with all NaN values
        extracted_data = extracted_data.dropna(how='all')
        if start_idx > 0:
            extracted_data.columns = extracted_data.iloc[0]
            extracted_data = extracted_data.iloc[1:]
            extracted_data.reset_index(drop=True, inplace=True)
        else:
            None

    for index, row in extracted_data.iterrows():
        v_name = None
        column_dict = dict()
        for column in extracted_data.columns:
            if column != 'name':
                if pd.notnull(row[column]):
                    value = row[column]
                    if isinstance(value, str) and value.startswith("[") and value.endswith("]"):
                        values = value[1:-1].split(', ')
                    else:
                        values = value
                    
                    column_dict[column] = values
            else:
                v_name = row[column]
        if '- name: "'+v_name+'"' in row_dict:
            None
        else:
            row_dict[ '- name: "'+v_name+'"' ] = list()
        row_dict[ '- name: "'+v_name+'"' ].append( column_dict )



0 8
11 19
22 24


In [4]:
data_dict = dict()

for v_name, v_name_list in row_dict.items():
    
    data_dict[v_name] = dict()

    
    for v_name_dict in v_name_list:
        
        if "Base_Category" in v_name_dict:
            v_base_category = v_name_dict["Base_Category"]
            if v_base_category in data_dict[v_name]:
                None
            else:
                data_dict[v_name][v_base_category] = list()

            blist = list()
            for bd_key, bd_val in v_name_dict.items():
                if bd_key in ("Base_Category", "Category"):
                    None
                else:
                    blist += ((bd_key,bd_val),)

            data_dict[v_name][v_base_category].append(blist)
        else:
            for mainkey, mainval in v_name_dict.items():
                data_dict[v_name][mainkey] = mainval

In [5]:
def Process_dictdata(dictdata, Final_Yml, tabbed_int):
    for dictkey, dictval in dictdata.items():
        #print (dictkey, dictval, type(dictval))
        if type(dictval) is list:
            Final_Yml += ( ("  "*tabbed_int)+str(dictkey)+":\n" )
            if all(isinstance(elem, str) for elem in dictval):
                #print ("\tA")
                tabbed_int+=1
                for list_elem in dictval:
                    Final_Yml += ( ("  "*tabbed_int)+"- "+str(list_elem)+"\n" )
                tabbed_int-=1
            elif all(isinstance(elem, list) for elem in dictval):
                #print ("\tB")
                
                for list_elem in dictval:
                    list_elem_cnt = 0
                    for tuple_val in list_elem:
                        tabbed_int+=1
                        if list_elem_cnt == 0:
                            Final_Yml += ( ("  "*tabbed_int)+"- "+str(tuple_val[0])+": "+str(tuple_val[1])+"\n" )
                            tabbed_int+=1
                        else:
                            Final_Yml += ( ("  "*tabbed_int)+str(tuple_val[0])+": "+str(tuple_val[1])+"\n" )
                        list_elem_cnt+=1
                        tabbed_int-=1
                    tabbed_int-=1
            elif all(isinstance(elem, list) for elem in dictval):
                print ("\t\t\t","XXXXX",dictkey, dictval, type(dictval) )
        else:
            #print ("\tC")
            Final_Yml += ( ("  "*tabbed_int)+str(dictkey)+": "+str(dictval)+"\n" )
        #print ("\n\n")
    return Final_Yml

In [6]:
Final_Yml = ""
tabbed_int = 0
for data_key, dictdata in data_dict.items():
    Final_Yml += data_key+"\n"
    tabbed_int+=1
    Final_Yml = Process_dictdata(dictdata, Final_Yml, tabbed_int)
    tabbed_int-=1
print (Final_Yml)

- name: "AISONLINE"
  count: 5
  target:
    - SG
    - HK
  target-zone: dmz3
  os: SCB-RHEL8
  cpu: 32
  memory: 64
  san:
    - size: 200
      index: 1
      scsi-controller: 0
      mount-path: /app/instancel
    - size: 700
      index: 1
      scsi-controller: 0
      mount-path: /app/batch
- name: "BATCH1"
  count: 7
  target:
    - SG
    - HK
  target-zone: dmz3
  cpu: 8
  memory: 32
  OS: SCB-RHEL8
  san:
    - size: 200
      index: 2
      scsi-controller: 0
      mount-path: /app/instance2
    - size: 10
      index: 2
      scsi-controller: 0
      mount-path: /u01/app/oracle
- name: "ZZZtemp"
  count: 5
  target:
    - SG
    - HK
  target-zone: dmz3
  os: SCB-RHEL8
  cpu: 64
  memory: 256
  nas:
    - nas-size: 200
      nas-index: 1
      nas-scsi-controller: 0
      nas-mount-path: /app/instancel
    - nas-size: 700
      nas-index: 1
      nas-scsi-controller: 0
      nas-mount-path: /app/batch

